<div align=center> <font color="#3C5BDA" size=8 face="微软雅黑"> Arrival Time Prediction using a Stacked Model </font> </div>

### Table of Contents:
- 1. Import Dependencies
- 2. Exploratory Data Analysis (EDA) and Data Preprocessing
    - Loading and Processing flight_data.csv and JFK NYC LGA Weather.csv
    - Creating, Loading and Processing JFK EWR and LGA Flights.csv
- 3. Data Modeling and Evaluation
    - Creating a XGBoost Regressor
    - Creating a CatBoost Regressor
    - Evaluating the CatBoost and XGBoost Regressor
    - Creating a Stacked Model
    - Evaluating the Stacked Model
- 4. Conclusion

<div align=center> <font color="#3C5BDA" size= face="1. Import Dependencies"> Arrival Time Prediction using a Stacked Model </font> </div>

In [1]:
import pandas as pd